### 데이터 전처리

✔️ 데이터 분석
   - 결측치 체크 및 처리
   - 중복 체크 및 처리
   - 이상치 체크 및 처리
   - 데이터 타입 정리  => (실제 값 2.39              --- string 데이터 타입)
   - 데이터 타입 변환 => (실제 값 32, 21, 9, 3, ... --- 범주형 데이터 타입)
   - 데이터 인코딩 처리 => (문자형 데이터 ---> 숫자형)
   - 데이터 스케일링 처리 => ( 수치형 데이터 ---> 값의 범위 일치)

In [1]:
# Module Loading
import pandas as pd
from numpy import nan

In [2]:
# Create Data
names = ['홍길동', '이나영', '마징가', '변사또', '원더우먼', '원빈', '현빈', '박보검', '김상민', '김한림']
ages = [0, 4, 21, 20, 38, 100, 40, 61, 11, 23]
genders = ['M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M']

In [3]:
personDF = pd.DataFrame({'name':names, 'age':ages, 'gender':genders})

In [4]:
personDF.head(3)

,name,age,gender
0,홍길동,0,M
1,이나영,4,F
2,마징가,21,M


In [5]:
personDF.dtypes

name      object
age        int64
gender    object
dtype: object

#### [데이터 전처리]
(1) 데이터 타입 체크 및 처리

✔️ astype('데이터 타입') : 반드시 원본 저장해야 적용 됨. inplace 매개변수 ❌

In [6]:
## "gender" columns (object ===> category)
personDF.gender = personDF.gender.astype('category')

(2) 연속형 데이터 => 범주형 데이터로 변환
- astype('데이터타입') => 대량 범주형 생성 ❗문제❗
- cut(데이터, 구간정보, 구간라벨) => 범위 즉 구간 설정 => 범주로 변경

In [7]:
# personDF.age ----> 연령대 범주형
# 아동기, 청년, 중년, 장년, 노년

- pd.cut() default result
```
✔️ a < ~ <= b
```

In [8]:
_bins = [0, 20, 40, 60, 80, 100]
ret1, bins = pd.cut(personDF.age, bins = _bins, labels = ['아동기', '청년', '중년', '장년', '노년'],
             right = True, retbins = True, include_lowest = True)

✔️include_lowest = True
```
0 <= ~ <= 20

20 < ~ <= 40

    .
    .
    .
```

In [9]:
ret1
# ages = [0, 4, 21, 20, 38, 100, 40, 61, 11, 23]

0    아동기
1    아동기
2     청년
3    아동기
4     청년
5     노년
6     청년
7     장년
8    아동기
9     청년
Name: age, dtype: category
Categories (5, object): ['아동기' < '청년' < '중년' < '장년' < '노년']

In [10]:
# 나이 카테고리 컬럼 추가
personDF['cate_age'] = ret1

In [11]:
bins

array([  0,  20,  40,  60,  80, 100])

In [12]:
personDF.head(3)

,name,age,gender,cate_age
0,홍길동,0,M,아동기
1,이나영,4,F,아동기
2,마징가,21,M,청년


In [13]:
personDF.dtypes

name          object
age            int64
gender      category
cate_age    category
dtype: object

### One-Hot-Encoding으로 변환
- 범주형 데이터를 0과 1로 구성된 패턴으로 인코딩
- 범주형 데이터 수만큼 자릿수 생성
- 해당 값에만 1을 세팅
- 함수 : pandas.get_dummies()

In [35]:
p1 = pd.get_dummies(personDF.cate_age)
p1.head(3)

,아동기,청년,중년,장년,노년
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0


In [34]:
p2 = pd.concat([personDF, p1], axis=1)
p2.head(3)

,name,age,gender,cate_age,아동기,청년,중년,장년,노년
0,홍길동,0,M,아동기,1,0,0,0,0
1,이나영,4,F,아동기,1,0,0,0,0
2,마징가,21,M,청년,0,1,0,0,0


In [30]:
sel = p2.loc[0, '아동기':]

In [31]:
sel.to_list()

[1, 0, 0, 0, 0]

In [32]:
sel.to_numpy()

array([1, 0, 0, 0, 0], dtype=object)

### scikit-learn 사용 문자열 범주형 인코딩

In [36]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

In [37]:
# Encoder 객체 생성
le = LabelEncoder()

In [38]:
# 전달된 데이터 확인 후 라벨 생성
le.fit(personDF.cate_age)

LabelEncoder()

In [39]:
le.classes_

array(['노년', '아동기', '장년', '청년'], dtype=object)

In [44]:
# 정수로 인코딩
en_age = le.transform(personDF.cate_age)
en_age

array([1, 1, 3, 1, 3, 0, 3, 2, 1, 3])

In [53]:
# 정수 => 문자열 되돌리기
le.inverse_transform(en_age)

array(['아동기', '아동기', '청년', '아동기', '청년', '노년', '청년', '장년', '아동기', '청년'],
      dtype=object)

In [54]:
## One-Hot-Encoder로 변환
ohe = OneHotEncoder(sparse = True)

In [55]:
ohe.fit(personDF.cate_age.to_numpy().reshape(-1, 1))
        # 2차원 형태!

OneHotEncoder()

In [56]:
ohe.transform(personDF.cate_age.to_numpy().reshape(-1, 1)) #.toarrayay()

<10x4 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>